# Apilamiento de productos  GeoTiff en pilas de datos de series temporales

Autor: Josef Kellndorfer, Earth Big Data LLC

Adaptado por: Cristhian Forero para la investigación: Diseño de una propuesta de optimización del sistema de alertas tempranas por deforestación para Colombia, caso de estudio “Corazón de la Amazonia”

Este cuaderno describe cómo apilar datos de Sentinel-1 en pilas de datos de series temporales con tablas ráster virtuales. Estos portátiles requieren que GDAL esté instalado y que el ejecutable **gdalbuildvrt** esté en la ruta del sistema.

Algunas suposiciones:

1) Este cuaderno asume que todos los datos que se apilarán se procesan previamente en la misma proyección, el espacio entre píxeles y las coordenadas de las esquinas son múltiplos sin fracciones del espacio entre píxeles.  

2) Convenciones de nomenclatura: los nombres de archivo geotiff contienen información sobre las tomas de datos, incluida la fecha y la ruta, separados por un guión bajo.

3) Si se apilan datos con diferentes ubicaciones en las esquinas, la extensión VRT será la extensión máxima de todos los archivos apilados. 

SALIDA:

- pilas vrt para cada directorio RTC, para cada ruta y para las pilas combinadas. FINAL: .vrt
- archivos de fecha correspondientes con entradas "AAAAMMDD" por línea por fecha. FINALIZACIÓN: .fechas

In [1]:
# Import python modules
import os,sys
import datetime
import glob
import subprocess as sp
import tempfile
import gdal

In [23]:
# Establezca DryRun en "True" para obtener una lista de los comandos gdalbuildvrt solo sin realizar la calibración
DryRun=True
# Proporcione el directorio raíz donde se almacenan los subdirectorios que contienen los archivos RTC
rootdir='F:/Cristhian/Tesis_capitulo_2/Imagenes_areasdeanalisis_marginaldelaselva/Landsat_Sentinel/2_Norm_con_info/'
# Proporcionar una lista de directorios RTC relativos al rootdir
rtcdirs=['TODAS',]
# Dar una extensión objetivo en coordenadas de proyección
targetextent=None

## Defina una función para encontrar todos los archivos en un directorio RTC, extraiga fechas y rutas y llame a gdalbuildvrt

In [24]:
def make_timeseries_vrt(rootdir,subdir,targetextent=None,prefix='',DryRun=False):
  
  # Obtenga el directorio de trabajo actual y cambie al directorio raíz
    cwd=os.getcwd()
    os.chdir(rootdir)

    imagelist = [x for x in glob.glob(os.path.join(subdir,'*.tif'))]
   # Obtener índice para una fecha válida en la lista separada por '_'
    first=imagelist[0].split('_')
    for i in range(len(first)):
        try:
            datetime.datetime.strptime(first[i],'%Y%m%d')
            idx=i
        except ValueError:
            next
    # Ahora obtén las fechas de todos los archivos
    dates=[x.split('_')[idx] for x in imagelist]
    # Ordenar por fechas y reorganizar la lista de imágenes en consecuencia
    datessorted=dates.copy()
    datessorted.sort()
    indices=[dates.index(x) for x in datessorted]
    imagelistsorted=[imagelist[x] for x in indices]

    # Obtener todos los caminos distintos
    paths=[x.split('_')[idx+1] for x in imagelistsorted]
    distinct_paths=set(paths)

    # We make a directory with vrtnames and have filelists as the entry
    vrtnames={}

    # Crea un vrtname para toda la pila. Usamos el nombre del subdirectorio como raíz para el archivo vrt:
    vrtname=prefix+subdir+'.vrt'
#     vrtnames[vrtname]={'filenames':imagelist, 'dates':dates}

    vrtnames[vrtname]={'filenames':imagelistsorted, 'dates':datessorted}


# Si hay más de una ruta, las agregamos al diccionario
     # para hacer vrtnames y listas correspondientes para archivos y fechas por ruta:
    if len(distinct_paths) > 1:
        for p in distinct_paths:
            # Obtenga el subconjunto de la lista de imágenes que corresponde a esta ruta:
            imagelist_subset=[x for x in imagelistsorted if x.find('_'+p+'_')>-1]
            dates_subset=[x.split('_')[idx] for x in imagelist_subset]
            # El nombre VRT se construye a partir del primer nombre de archivo en la lista rtc sin el 
            vrtname=prefix+subdir+'_'+p+'.vrt'
            vrtnames[vrtname]={'filenames':imagelist_subset, 'dates':dates_subset}

    # Ahora construye las vrts:
    if targetextent==None:
        te=''
    else:
        te='-te '+ targetextent

    for i in vrtnames:
        tfile=tempfile.NamedTemporaryFile(mode='w+',delete=False)
        for j in vrtnames[i]['filenames']:
            tfile.write(j+'\n')
        tfile.close()
        cmd='gdalbuildvrt -separate -allow_projection_difference {} -input_file_list {} -overwrite {}'.format(te,tfile.name,i)
        if not DryRun:
            sp.call(cmd.split())
            # Escribir los archivos de fechas
            datefilename=i.replace('.vrt','.dates')
            with open(datefilename,"w+") as f:
                for j in vrtnames[i]['dates']:
                    f.write(j+'\n')
            # Establecer los nombres de las bandas como fechas
            img_handle=gdal.Open(i)
            for j in range(img_handle.RasterCount):
                x=vrtnames[i]['dates'][j]
                # formatear de YYYYMMDD a YYYY-MM-DD
                date=x[:4]+'-'+x[4:6]+'-'+x[6:8]
                # Establecer los metadatos y la descripción en la fecha
                band=img_handle.GetRasterBand(j+1)
                band.SetMetadataItem("Date",date)
                band.SetDescription(date)
            del img_handle
        else:
            print('DryRun:',end=' ')
        print(cmd)

        os.remove(tfile.name)
    # Cambiar de nuevo al directorio de trabajo original
    os.chdir(cwd)

In [25]:
## print (imagelist)

## Ahora recorra todos los subdirectorios para hacer los vrts

Recorremos todos los subdirectorios (rtcdirs) para generar los vrts. Cambie el prefijo o DryRun para la prueba,

       make_timeseries_vrt(rootdir,i,prefix='test_',DryRun=False)  
       make_timeseries_vrt(rootdir,i,prefix='',DryRun=True)

In [26]:
for i in rtcdirs:
    make_timeseries_vrt(rootdir,i,prefix='test_',DryRun=False)

gdalbuildvrt -separate -allow_projection_difference  -input_file_list C:\Users\crist\AppData\Local\Temp\tmpe9cc_wz0 -overwrite test_TODAS.vrt


Ahora podemos inspeccionar esos archivos recién creados con gdal

In [27]:
os.chdir(rootdir)

In [28]:
print(gdal.Info('test_NIR.vrt'))

Driver: VRT/Virtual Raster
Files: test_NIR.vrt
       NIR\s2_t18nxh_20190101_nir.tif
       NIR\s2_t18nxh_20190106_nir.tif
       NIR\lc08_pat759_20190108_nir.tif
       NIR\s2_t18nxh_20190210_nir.tif
       NIR\s2_t18nxh_20190215_nir.tif
       NIR\s2_t18nxh_20190220_nir.tif
       NIR\lc08_pat759_20190225_nir.tif
       NIR\s2_t18nxh_20190327_nir.tif
       NIR\s2_t18nxh_20190401_nir.tif
       NIR\le07_pat759_20190406_nir.tif
       NIR\s2_t18nxh_20190426_nir.tif
       NIR\s2_t18nxh_20190501_nir.tif
       NIR\s2_t18nxh_20190511_nir.tif
       NIR\s2_t18nxh_20190526_nir.tif
       NIR\s2_t18nxh_20190531_nir.tif
       NIR\s2_t18nxh_20190605_nir.tif
       NIR\le07_pat759_20190625_nir.tif
       NIR\s2_t18nxh_20190715_nir.tif
       NIR\s2_t18nxh_20190809_nir.tif
       NIR\le07_pat759_20190812_nir.tif
       NIR\lc08_pat759_20190820_nir.tif
       NIR\s2_t18nxh_20190824_nir.tif
       NIR\s2_t18nxh_20190829_nir.tif
       NIR\s2_t18nxh_20190903_nir.tif
       NIR\s2_t18nxh_20190908

In [29]:
rootdir

'F:/Cristhian/Tesis_capitulo_2/Imagenes_areasdeanalisis_marginaldelaselva/Landsat_Sentinel/2_Norm_con_info/'